In [2]:
import torch
import torch.nn as nn
from diffusers.schedulers import PNDMScheduler
import onnx
import onnxruntime
from torch.onnx import export
from pathlib import Path
from packaging import version
from diffusers import DiffusionPipeline
import os
import shutil
from typing import List, Optional, Tuple, Union
import numpy as np
import inspect
from tqdm import tqdm

import gc
gc.collect()

0

In [3]:
device = "cpu"
dtype = torch.float32

In [4]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float32)

---
#### Text Embedding


In [5]:
text_input = pipeline.tokenizer(
    ["A smile cat"],
    padding="max_length",
    max_length=pipeline.tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt",
    ).input_ids.to(device=device, dtype=torch.int32)

In [8]:
ort = onnxruntime.InferenceSession('./onnx-models/TextEmbedding/model.onnx')
out_t_embed = ort.run(None, {'prompt': text_input.detach().cpu().numpy()})[0]
out_t_embed.shape

(2, 77, 768)

---
### Denoising

In [ ]:
ort_de = onnxruntime.InferenceSession('./onnx-models/Denoising/model.onnx')

In [ ]:
out_de = ort_de.run(None, {'encoder_hidden_states': out_t_embed})[0]
out_de.shape

---
### Decoder

In [ ]:
from PIL import Image
def numpy_to_pil(images):
    """
    Convert a numpy image or a batch of images to a PIL image.
    """
    if images.ndim == 3:
        images = images[None, ...]
    images = (images * 255).round().astype("uint8")
    if images.shape[-1] == 1:
        # special case for grayscale (single channel) images
        pil_images = [Image.fromarray(image.squeeze(), mode="L") for image in images]
    else:
        pil_images = [Image.fromarray(image) for image in images]

    return pil_images

In [ ]:
ort_VaeDecoder = onnxruntime.InferenceSession('./onnx-models/Decoder/model.onnx')

In [ ]:
out = ort_VaeDecoder.run(None, {'latent_sample' : out_de.astype(np.float16)})[0]
out.shape

In [ ]:
img = numpy_to_pil(out)
img[0].show()